# `data.table`: Where Have You Been All My Life? 

# °₊·ˈ∗♡( ˃̶᷇ ‧̫ ˂̶᷆ )♡∗ˈ‧₊°

`data.table` is an easy to use R package for querying and manipulating data. Dataset updates run much faster than with the base R paradigm, and specifying complicated calculations is more intuitive. This talk will show some of the package's features as well as point out a couple of gotchas.

* Add/update/delete columns by reference by group using no copies at all

* Really simplified data manipulation for me, especially on longitudinal data.

* Focus on the add/update/delete columns and indexing

* Many more capabilities not covered here!

In [ ]:
setwd("/Users/ruddjm/Dropbox/miscWork/presentations/dataDotTable")
library(data.table)
library(rbenchmark)
library(Hmisc)
options(repr.matrix.max.rows = 40, repr.matrix.max.cols = 10)

## The data.table class

* data.table is a package, but it's also a **class**.
* 'Enhanced data.frame'
* You can turn a data frame into a data table, and it will still do all the things data frames do.

### Let's make a data.table

* Need to get your data in a data.table to use data.table's capabilities
* How to turn a data frame into a data.table
* Can also read in using a data.table function that automatically makes it a data.table

* Can use `data.table` function with columns input directly 

In [9]:
DT = data.table(a=LETTERS[c(1,1:3)],b=4:7,key="a")
DT

a,b
A,4
A,5
B,6
C,7


In [10]:
class(DT)

[1] "data.table" "data.frame"

Let's "unmake" it a data.table

In [11]:
class(DT) = 'data.frame'

And back to a data.table

In [12]:
DT = data.table(DT)

## Example data: plants

In [3]:
#http://plants.usda.gov/adv_search.html
plantsDF = read.csv(file = 'exampleData/usdaQuery.txt',
  stringsAsFactors = FALSE,
  na.strings = c("", "NA"))

In [4]:
head(plantsDF)

Accepted.Symbol,Synonym.Symbol,Scientific.Name,State.and.Province,Duration,⋯,Foliage.Color,Height..Mature..feet.,Drought.Tolerance,Shade.Tolerance,Bloom.Period
ABAM,NA,Abies amabilis,"USA (AK, CA, OR, WA), CAN (BC)",Perennial,⋯,Gray-Green,165,Low,Tolerant,Late Spring
ABBA,NA,Abies balsamea,"USA (CT, IA, IN, MA, MD, ME, MI, MN, NH, NJ, NY, OH, PA, RI, VA, VT, WI, WV), CAN (AB, LB, MB, NB, NF, NS, ON, PE, QC, SK), FRA (SPM)",Perennial,⋯,Green,60,Low,Tolerant,Mid Summer
ABCO,NA,Abies concolor,"USA (AZ, CO, ID, MA, ME, NM, NV, OR, UT, WY)",Perennial,⋯,Green,120,Medium,Intermediate,Late Spring
ABFR,NA,Abies fraseri,"USA (GA, NC, TN, VA)",Perennial,⋯,Dark Green,55,Low,Tolerant,Mid Spring
ABGR,NA,Abies grandis,"USA (CA, ID, MT, OR, WA), CAN (BC)",Perennial,⋯,Green,150,Medium,Tolerant,Late Spring
ABLA,NA,Abies lasiocarpa,"USA (AK, AZ, CA, CO, ID, MT, NM, NV, OR, UT, WA, WY), CAN (AB, BC, NT, YT)",Perennial,⋯,Green,90,Low,Tolerant,Late Spring


In [5]:
class(plantsDF)

[1] "data.frame"

In [6]:
plants = data.table(plantsDF)

In [7]:
class(plants)

[1] "data.table" "data.frame"

In [8]:
names(plants) = gsub('\\.', '', names(plants))
names(plants)

[1] "AcceptedSymbol"    "SynonymSymbol"     "ScientificName"   
 [4] "StateandProvince"  "Duration"          "GrowthHabit"      
 [7] "NativeStatus"      "FlowerColor"       "FlowerConspicuous"
[10] "FoliageColor"      "HeightMaturefeet"  "DroughtTolerance" 
[13] "ShadeTolerance"    "BloomPeriod"

## Indexing

### Rows

Can subset rows using
* row indices
* logical statements
* logical statements don't need to refer to the df name!

In [9]:
# First row:
plants[1, ]
# or
plants[1]

AcceptedSymbol,SynonymSymbol,ScientificName,StateandProvince,Duration,⋯,FoliageColor,HeightMaturefeet,DroughtTolerance,ShadeTolerance,BloomPeriod
ABAM,NA,Abies amabilis,"USA (AK, CA, OR, WA), CAN (BC)",Perennial,⋯,Gray-Green,165,Low,Tolerant,Late Spring


AcceptedSymbol,SynonymSymbol,ScientificName,StateandProvince,Duration,⋯,FoliageColor,HeightMaturefeet,DroughtTolerance,ShadeTolerance,BloomPeriod
ABAM,NA,Abies amabilis,"USA (AK, CA, OR, WA), CAN (BC)",Perennial,⋯,Gray-Green,165,Low,Tolerant,Late Spring


Get all the plants with white-gray foliage:

In [11]:
head(plants[FoliageColor == 'White-Gray', ])

AcceptedSymbol,SynonymSymbol,ScientificName,StateandProvince,Duration,⋯,FoliageColor,HeightMaturefeet,DroughtTolerance,ShadeTolerance,BloomPeriod
ARLU,NA,Artemisia ludoviciana,"USA (AR, AZ, CA, CO, CT, DE, FL, GA, IA, ID, IL, IN, KS, KY, LA, MA, MD, ME, MI, MN, MO, MS, MT, NC, ND, NE, NH, NJ, NM, NV, NY, OH, OK, OR, PA, RI, SC, SD, TN, TX, UT, VA, VT, WA, WI, WY), CAN (AB, BC, MB, NB, NT, ON, PE, QC, SK)",Perennial,⋯,White-Gray,3.0,High,Intolerant,Early Spring
ARRI2,NA,Artemisia rigida,"USA (ID, MT, OR, WA)",Perennial,⋯,White-Gray,2.0,High,Intolerant,Late Summer
CORA,NA,Coleogyne ramosissima,"USA (AZ, CA, CO, NV, UT)",Perennial,⋯,White-Gray,6.0,High,Intolerant,Mid Spring
COFL3,NA,Corydalis flavula,"USA (AL, AR, CT, DC, DE, FL, GA, IA, IL, IN, KS, KY, LA, MD, MI, MO, MS, NC, NE, NJ, NY, OH, OK, PA, SC, TN, VA, WV), CAN (ON)",Annual,⋯,White-Gray,1.5,Low,Intermediate,Mid Spring
CRMO4,NA,Crepis modocensis,"USA (CA, CO, ID, MT, NV, OR, UT, WA, WY), CAN (BC)",Perennial,⋯,White-Gray,1.4,Medium,Intolerant,Late Spring
ELCO,NA,Elaeagnus commutata,"USA (AK, CO, ID, KY, MD, MN, MT, ND, NY, RI, SD, TX, UT, WA, WY), CAN (AB, BC, MB, NT, NU, ON, QC, SK, YT)",Perennial,⋯,White-Gray,12.0,High,Intolerant,Early Summer


## Columns

* no quotes needed on column names
* wrap names in `.()`

In [13]:
head(plants[ , .(BloomPeriod)])

BloomPeriod
Late Spring
Mid Summer
Late Spring
Mid Spring
Late Spring
Late Spring


* Can also omit the `.()`
* Result will be a vector, not a data.table

In [14]:
head(plants[ , BloomPeriod])

[1] "Late Spring" "Mid Summer"  "Late Spring" "Mid Spring"  "Late Spring"
[6] "Late Spring"

Select more than one column:

In [15]:
head(plants[ , .(FlowerColor, Duration)])

FlowerColor,Duration
Yellow,Perennial
Yellow,Perennial
Red,Perennial
Purple,Perennial
NA,Perennial
NA,Perennial


What about indexing by the column number?

In [24]:
which(names(plants) == 'BloomPeriod')

[1] 14

In [16]:
head(plantsDF[ , 14])

[1] "Late Spring" "Mid Summer"  "Late Spring" "Mid Spring"  "Late Spring"
[6] "Late Spring"

In [26]:
plants[ , 14]

[1] 14

# ఠ ͟ಠ

**Must use `with` argument**

In [17]:
head(plants[ , 14, with = FALSE])

BloomPeriod
Late Spring
Mid Summer
Late Spring
Mid Spring
Late Spring
Late Spring


What about indexing by the **string** column name?

In [28]:
plants[ , 'BloomPeriod'] # doesn't work

[1] "BloomPeriod"

In [29]:
plants[ , .('BloomPeriod')] # doesn't work

V1
BloomPeriod


Again, need to use `with`

In [18]:
head(plants[ , 'BloomPeriod', with = FALSE])

BloomPeriod
Late Spring
Mid Summer
Late Spring
Mid Spring
Late Spring
Late Spring


### This may break your existing code!

### Indexing operations my return a data.table or a vector, depending on the way you index 
Investigate the object returned by different indexing.

In [ ]:
DT = data.table(a=LETTERS[c(1,1:3)],b=4:7,key="a")
DF = data.frame(a=LETTERS[c(1,1:3)], b=4:7, stringsAsFactors = FALSE)
DT[, a]
DT[, "a", with = FALSE]


# Using the column name with no quotes gives a vector.
str(DT[, a])
# chr [1:4] "A" "A" "B" "C"

# Using the CHARACTER NAME returns a data frame (thats also a data.table).
str(DT[, "a", with = FALSE])
#Classes ‘data.table’ and 'data.frame':  4 obs. of  1 variable:
# $ a: chr  "A" "A" "B" "C"
# - attr(*, "sorted")= chr "a"
# - attr(*, ".internal.selfref")=<externalptr>

str(DT[, a])
str(DT[, 1, with = FALSE])
str(DT[, "a", with = FALSE])
str(DT[["a"]])
str(DT[[1]])
DT[, 1]                  # NO!
DT[, .(1)]               # NO!
DT[, 1, with = FALSE]       # returns a data table
DT[, .(a)]
DT[ , list(a)]


# data FRAME returns a vector for all these uses.
str(DF[["a"]])
str(DF[[1]])
str(DF[, 1])
str(DF[, "a"])

## How to access help files about indexing

* Documentation for indexing
* Tricky because we use the `[` operator and not a function name.

In [31]:
#help('[.data.frame')
#help('[.data.table')
?'[.data.table'
# Note the quote

## Can call all sorts of functions within the brackets! 
* Inside or outside!
* Kind of like using the `with` function

# ✧٩(•́⌄•́๑)و ✧

In [32]:
plants[ , head(HeightMaturefeet)]

[1] 165  60 120  55 150  90

In [33]:
plants[ , table(BloomPeriod, FlowerColor)]

               FlowerColor
BloomPeriod     Blue Brown Green Orange Purple Red White Yellow
  Early Spring     4     1     8      4     18   5    36     51
  Early Summer     4     1    10      0     12   8    38     50
  Fall             1     0     5      0      2   3     3      7
  Indeterminate    2     0     8      0      5   4    13     18
  Late Spring     11    14   105      4     53  25   110    140
  Late Summer      5     4    27      0     20   4    22     45
  Late Winter      0     2     0      0      0   0     3      6
  Mid Spring      11     4    34      4     24  16    68    112
  Mid Summer       4     3    26      1     21   2    24     44
  Spring           7     2    37      1     11  13    42     43
  Summer          10     9    32      1     17  10    30     34
  Winter           0     0     0      0      0   1     3      2

## Calculations **BY** another variable

Get the mean height by growth habit.

In [34]:
plants[ , mean(HeightMaturefeet, na.rm = TRUE), by = GrowthHabit]  #[ , 2, with = FALSE]

GrowthHabit,V1
Tree,87.977143
NA,19.958000
Forb/herb,2.741739
"Tree, Shrub, Vine",18.000000
"Tree, Shrub",28.176355
Graminoid,2.929167
Shrub,7.545324
"Subshrub, Forb/herb",2.962821
"Subshrub, Shrub, Forb/herb",1.900000
"Forb/herb, Shrub, Subshrub",1.200000


## Assignment/Creating new variables

* Much faster

In [35]:
# DF$c <- 18:13 # results in a deep copy of the entire data frame.

### `data.table` allows you to add/update/delete columns by reference using no copies at all

* Use the `:=` operator

Let's make a new variable that is the plant height plus 1.

*Normally, these in-place assignments do not produce any output. They are producing output here. It may be an issue with jupyter notebook?*

In [ ]:
head(plants[ , oneFootTaller := HeightMaturefeet + 1])

In [20]:
head(plants[ , .(ScientificName, HeightMaturefeet, oneFootTaller)])

ScientificName,HeightMaturefeet,oneFootTaller
Abies amabilis,165,166
Abies balsamea,60,61
Abies concolor,120,121
Abies fraseri,55,56
Abies grandis,150,151
Abies lasiocarpa,90,91


Make a new variable and initialize it to missing:

In [ ]:
plants[ , numberOfPetals := NA]

### Can update a specific value

In [39]:
# Let's update the petal count for one particular plant.
# Let's first find the scientific name for the 246th plant:
plants[246, ScientificName]

[1] "Schoenoplectus maritimus"

In [ ]:
plants[ScientificName == "Schoenoplectus maritimus", numberOfPetals := 17]

In [41]:
plants[240:250, .(ScientificName, numberOfPetals)]

ScientificName,numberOfPetals
Blechnum spicant,NA
Blepharoneuron tricholepis,NA
Boehmeria cylindrica,NA
Bolboschoenus fluviatilis,NA
Schoenoplectus fluviatilis,NA
Bolboschoenus maritimus,NA
Schoenoplectus maritimus,TRUE
Bolboschoenus robustus,NA
Schoenoplectus robustus,NA
Boltonia asteroides,NA


Look at the value of numberOfPetals for "Schoenoplectus fluviatilis!" It's not 17 like we assigned!

The reason is that when whe created a new variable and set it to missing, it automatically made the new variable a *logical* type.

In [42]:
typeof(plants$numberOfPetals)
str(plants$numberOfPetals)

[1] "logical"

 logi [1:1725] NA NA NA NA NA NA ...


If we want the new variable to contain integers, we can define it initially by explicitly setting the type with `as.vector(NA, mode = 'integer')`.

In [ ]:
# Start by removing the previous version of numberOfPetals from the DT:
plants[ , numberOfPetals := NULL]
plants[ , numberOfPetals := as.vector(NA, mode = 'integer')]

In [44]:
typeof(plants$numberOfPetals)
str(plants$numberOfPetals)

[1] "integer"

 int [1:1725] NA NA NA NA NA NA NA NA NA NA ...


## To see the documentationfor assignment, search for ":="
?":="

### Removing columns

In [ ]:
plants[ , c('AcceptedSymbol', 'SynonymSymbol', 'NativeStatus') := NULL]

This changes the plants data frame *in place*.

### Assignment BY another variable

Let's add a new column with this mean height, according to the plant's growth habit.

In [ ]:
plants[ , heightClassMean := mean(HeightMaturefeet, na.rm = TRUE), by = GrowthHabit]

In [28]:
head(plants[ , .(ScientificName, HeightMaturefeet, GrowthHabit, heightClassMean)][order(c(GrowthHabit))])

ScientificName,HeightMaturefeet,GrowthHabit,heightClassMean
Acalypha virginica,3.0,Forb/herb,2.741739
Achillea millefolium,3.0,Forb/herb,2.741739
Achillea millefolium var. occidentalis,3.0,Forb/herb,2.741739
Acmella oppositifolia var. repens,0.7,Forb/herb,2.741739
Aconitum columbianum,5.0,Forb/herb,2.741739
Acorus americanus,3.9,Forb/herb,2.741739


## Another example data set

* Longitudinal
* Demonstrate some practical applications
* Can download this data at https://www.r-project.org/nosvn/conferences/useR-2007/competition/ragged.zip

In [23]:
load(file = 'exampleData/ragged.Rdata', verbose = TRUE)
head(ragged[order(ragged$id), ])

Loading objects:
  ragged


id,futime,status,sex,agebl,⋯,alkphos,ALT,protime,platelets,stage
1,400,dead,female,21464,⋯,1718,138.0,190,12.2,4
1,400,dead,female,21464,⋯,1612,6.2,183,11.2,4
2,5169,pretx,female,20617,⋯,7395,113.5,221,10.6,3
2,5169,pretx,female,20617,⋯,2107,139.5,188,11.0,3
2,5169,pretx,female,20617,⋯,1711,144.2,161,11.6,3
2,5169,pretx,female,20617,⋯,1365,144.2,122,10.6,3


In [ ]:
ragged = data.table(ragged)
ragged[ , id := as.character(id)]

In [30]:
setkey(ragged, id, visittime)
key(ragged)

[1] "id"        "visittime"

We can query by id without stating 'id,' since id is set as a key. `visittime` is set as a secondary key. 

Get the row for id 99 and visittime 193.

In [50]:
ragged[.('99', 193)]

id,futime,status,sex,agebl,⋯,alkphos,ALT,protime,platelets,stage
99,4489,pretx,male,17703,⋯,231,21.7,218,10.1,2


Calculate each person's follow up time by taking their highest visittime. Accomplish this by using max(visittime) with by = id.

In [31]:
head(ragged[ ,  max(visittime), by = id])

id,V1
1,192
10,0
100,514
101,4049
102,3354
103,0


Suppose we need those values in our datasat. I would usually use merge(). 

The following code both calculates everyone's last visit time and merges the results back into the data. 

In [ ]:
# This is the coolest thing ever!!!
ragged[ ,  lastVisitTime := max(visittime), by = id]

In [36]:
ragged[400:430 , .(id, visittime, lastVisitTime)]

id,visittime,lastVisitTime
15,196,3537
15,366,3537
15,735,3537
15,1128,3537
15,1491,3537
15,1946,3537
15,2143,3537
15,2499,3537
15,2891,3537
15,3537,3537


Make a new variable indicating the number of times each person occurs in the data.

In [ ]:
ragged[ , numberObservations := .N, by = id]

In [39]:
head(ragged[ , .(id, visittime, numberObservations)], 30)

id,visittime,numberObservations
1,0,2
1,192,2
10,0,1
100,0,4
100,171,4
100,358,4
100,514,4
101,0,12
101,178,12
101,368,12


You can do calculations by two variables.

Get the mean platelets for each patient within each level of stage, and add that as a new variable

In [ ]:
ragged[ , patientMeanPlateletByStage := mean(platelets, na.rm = TRUE), by = .(id, stage)]

In [43]:
ragged[400:415, .(id, platelets, stage, patientMeanPlateletByStage)]

id,platelets,stage,patientMeanPlateletByStage
15,10.4,3,10.70000
15,10.4,4,10.62222
15,10.1,4,10.62222
15,10.0,4,10.62222
15,9.8,4,10.62222
15,10.4,4,10.62222
15,10.7,4,10.62222
15,10.5,4,10.62222
15,10.7,4,10.62222
15,13.0,4,10.62222


Make a new variable which gives each patient's immediately previous platelet measure. 

NB: The data must be sorted by time for this to work. How do we know that these data are already sorted by visittime?

In [ ]:
ragged[ , plateletsLag1 :=  shift(platelets), by = id]

In [45]:
ragged[650:670 , .(id, visittime, platelets, plateletsLag1)]

id,visittime,platelets,plateletsLag1
186,782,11.8,11.7
187,0,11.5,NA
187,217,11.5,11.5
187,356,11.1,11.5
187,729,20.8,11.1
188,0,11.0,NA
188,380,11.0,11.0
188,538,10.2,11.0
188,930,10.8,10.2
188,1324,10.3,10.8
